In [1]:
from time import time
import numpy as np
from matplotlib import pyplot as plt

# Qiskit
from qiskit import IBMQ, transpile, QuantumCircuit
from qiskit.providers.ibmq.managed import IBMQJobManager
from qiskit.quantum_info import partial_trace, Statevector, DensityMatrix
# Local modules
from utilities import startup, check_provider
from graphstate import GraphState, calc_negativities, calc_entanglement_entropy, plot_negativities_multi, calc_n_mean, plot_cxerr_corr

provider = startup()

Account enabled
Provider: ibm-q-melbourne


In [7]:
backend = provider.get_backend('ibm_perth')
graphstate = GraphState(backend, qubits_to_connect = [1,3,4,5,6])
graphstate.gen_full_qst_circuits()
print(len(graphstate.name_list))
#graphstate.full_qst_circuits['ZZ'].draw()
job = graphstate.run_full_qst_circuits(reps=1, shots=8192, qrem=True, sim='ideal')
print('job done')
#pvecs, counts = graphstate.full_qst_pvecs_from_result(job.result())
#print(pvecs)
#print(counts)
rho_list = graphstate.recon_full_density_mats(job.result(), apply_mit = False)
rho_list_mit = graphstate.recon_full_density_mats(job.result(), apply_mit = 'qrem')
print(rho_list)
print(rho_list_mit)
fidelities = graphstate.find_fidelities_to_GraphState(rho_list)
fidelities_mit = graphstate.find_fidelities_to_GraphState(rho_list_mit)
print(fidelities)
print(fidelities_mit)
#graphstate.full_qst_circuits['ZZZZ'].draw()

243
job done
[array([[ 0.03050232+0.j        ,  0.03159332-0.00069427j,
         0.03262329+0.00149536j, ..., -0.03085327-0.00048828j,
        -0.03157806+0.00225067j,  0.0295639 -0.00131226j],
       [ 0.03159332+0.00069427j,  0.0337677 +0.j        ,
         0.03121185+0.00016785j, ..., -0.03314972+0.00208282j,
        -0.03316498+0.00071716j,  0.0331192 +0.00084686j],
       [ 0.03262329-0.00149536j,  0.03121185-0.00016785j,
         0.03353882+0.j        , ..., -0.03134918-0.00042725j,
        -0.03101349+0.00197601j,  0.02946472-0.00048828j],
       ...,
       [-0.03085327+0.00048828j, -0.03314972-0.00208282j,
        -0.03134918+0.00042725j, ...,  0.03504944+0.j        ,
         0.03400421-0.00128174j, -0.03102112-0.00161743j],
       [-0.03157806-0.00225067j, -0.03316498-0.00071716j,
        -0.03101349-0.00197601j, ...,  0.03400421+0.00128174j,
         0.03158569+0.j        , -0.03180695-0.00191498j],
       [ 0.0295639 +0.00131226j,  0.0331192 -0.00084686j,
         0.02946

In [31]:
backend = provider.get_backend('ibm_perth')
test = GraphState(backend, qubits_to_connect = [1,2,3,5,6])

index_dict = {}
for i in range(len(test.qubits_to_connect)):
    index_dict[test.qubits_to_connect[i]] = test.nqubits-1-i
state = QuantumCircuit(test.nqubits)
state.h(list(range(test.nqubits)))
unconnected_edges = test.edge_list.copy()
print(unconnected_edges)
while unconnected_edges:
    connected_qubits = []  # Qubits already connected in the current time step
    connected_edges = []
    remove = []
    for edge in unconnected_edges:
        if np.any(np.isin(edge, connected_qubits)) == False:
            state.cz(index_dict[edge[0]], index_dict[edge[1]])
            connected_qubits.extend(edge)
            remove.append(edge)
    # Remove connected edges from unconnected edges list
    for edge in remove:
        unconnected_edges.remove(edge)

vec = Statevector(state)
rho_ideal = DensityMatrix(vec).data
print(rho_ideal)
state.draw()

[(1, 2), (1, 3), (3, 5), (5, 6)]
[[ 0.03125+0.j  0.03125+0.j  0.03125+0.j ...  0.03125+0.j -0.03125-0.j
   0.03125+0.j]
 [ 0.03125+0.j  0.03125+0.j  0.03125+0.j ...  0.03125+0.j -0.03125-0.j
   0.03125+0.j]
 [ 0.03125+0.j  0.03125+0.j  0.03125+0.j ...  0.03125+0.j -0.03125-0.j
   0.03125+0.j]
 ...
 [ 0.03125+0.j  0.03125+0.j  0.03125+0.j ...  0.03125+0.j -0.03125-0.j
   0.03125+0.j]
 [-0.03125+0.j -0.03125+0.j -0.03125+0.j ... -0.03125+0.j  0.03125+0.j
  -0.03125+0.j]
 [ 0.03125+0.j  0.03125+0.j  0.03125+0.j ...  0.03125+0.j -0.03125-0.j
   0.03125+0.j]]


┌───┐      
q_0: ┤ H ├────■─
     ├───┤    │ 
q_1: ┤ H ├─■──■─
     ├───┤ │    
q_2: ┤ H ├─■──■─
     ├───┤    │ 
q_3: ┤ H ├─■──┼─
     ├───┤ │  │ 
q_4: ┤ H ├─■──■─
     └───┘

In [30]:
state = QuantumCircuit(2)
state.h(0)
vec = Statevector(state)
print(vec.data)
rho_ideal = DensityMatrix(vec).data
print(rho_ideal)
state.draw()

[0.70710678+0.j 0.70710678+0.j 0.        +0.j 0.        +0.j]
[[0.5+0.j 0.5+0.j 0. +0.j 0. +0.j]
 [0.5+0.j 0.5+0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]]


┌───┐
q_0: ┤ H ├
     └───┘
q_1: ─────